In [ ]:
import pandas as pd
import os
import re
from pathlib import Path
from typing import List, Dict, Optional
import traceback

# ==============================================================================
# 1. 配置层 (Configuration Layer)
# ==============================================================================
class Config:
    """集中管理所有可配置参数"""
    INPUT_DIR = Path(r'E:\A智网\业扩分析\数据准备\新装增容数据')
    OUTPUT_FILE = Path(r'E:\A智网\业扩分析\数据准备\10kV以下业扩.xlsx')
    FILENAME_PATTERN = r'^\d{6}\.xlsx$'

    TARGET_SHEET_NAME = '国网湖北省电力有限公司'
    SKIP_ROWS = 3
    HEADER_ROWS_COUNT = 4

    # 核心指标的最终命名
    COL_TOTAL_CAPACITY = '完成新装增容_容量'
    COL_TOTAL_COUNT    = '完成新装增容_户数'
    COL_NEW_CAPACITY   = '完成新装_容量'
    COL_NEW_COUNT      = '完成新装_户数'
    COL_INCREASE_CAPACITY = '完成增容_容量'
    COL_INCREASE_COUNT    = '完成增容_户数'

    @staticmethod
    def get_sheets_to_create() -> Dict[str, str]:
        return {
            Config.COL_TOTAL_CAPACITY: Config.COL_TOTAL_CAPACITY,
            Config.COL_NEW_CAPACITY:   Config.COL_NEW_CAPACITY,
            Config.COL_INCREASE_CAPACITY: Config.COL_INCREASE_CAPACITY,
            Config.COL_TOTAL_COUNT:    Config.COL_TOTAL_COUNT,
            Config.COL_NEW_COUNT:      Config.COL_NEW_COUNT,
            Config.COL_INCREASE_COUNT: Config.COL_INCREASE_COUNT
        }

# ==============================================================================
# 2. 提取与解析层 (Extraction & Parsing Layer)
# ==============================================================================
def robust_to_numeric(series: pd.Series) -> pd.Series:
    """
    一个极其健壮的函数，用于清洗并转换一列可能包含文本格式数字的数据。
    它能处理千位分隔符(,)、首尾空格等常见问题。
    """
    # 步骤1: 移除所有千位分隔符 (,)
    # .str 访问器确保我们只在字符串上操作，对已经是数字的类型不会报错
    s = series.str.replace(',', '', regex=False)
    
    # 步骤2: 将清洗后的字符串转换为数字，无法转换的变成NaN
    return pd.to_numeric(s, errors='coerce')


def process_single_file(file_path: Path) -> Optional[pd.DataFrame]:
    """读取并处理单个Excel文件，返回一个干净的DataFrame"""
    year_month = file_path.stem
    print(f"  -> 正在处理文件: {file_path.name}")
    try:
        # !!! 终极修正 1: 强制将所有单元格作为字符串读取，避免pandas的自动类型推断 !!!
        df = pd.read_excel(
            file_path, sheet_name=Config.TARGET_SHEET_NAME,
            skiprows=Config.SKIP_ROWS + Config.HEADER_ROWS_COUNT,
            header=None,
            dtype=str  # <--- 这一行是解决问题的关键！
        )
        
        df.rename(columns={0: '序号', 1: '行业'}, inplace=True)

        df = df[df['行业'].astype(str).str.strip() != '栏目'].copy()
        
        df.dropna(how='all', subset=df.columns[2:], inplace=True)
        # 对序号列也使用健壮的转换
        df['序号'] = robust_to_numeric(df['序号'])
        df.dropna(subset=['序号'], inplace=True)
        df['序号'] = df['序号'].astype(int)
        df['年月'] = year_month
        
        df_final = df[['序号', '行业', '年月']].copy()
        
        # --- 终极修正 2：对所有目标列都应用我们强大的清洗函数 ---
        # 换算公式： Pandas索引 = 栏目号 + 1
        
        # 完成情况 - 合计
        df_final[Config.COL_TOTAL_COUNT]      = robust_to_numeric(df.iloc[:, 13+1]) # 栏目13
        df_final[Config.COL_TOTAL_CAPACITY]   = robust_to_numeric(df.iloc[:, 14+1]) # 栏目14

        # 完成情况 - 其中: (增容) - 10kV以
        df_final[Config.COL_INCREASE_COUNT]   = robust_to_numeric(df.iloc[:, 17+1]) # 栏目17
        df_final[Config.COL_INCREASE_CAPACITY]= robust_to_numeric(df.iloc[:, 18+1]) # 栏目18
        
        # 完成情况 - 其中:新装 - 10kV以下
        df_final[Config.COL_NEW_COUNT]        = robust_to_numeric(df.iloc[:, 23+1]) # 栏目23
        df_final[Config.COL_NEW_CAPACITY]     = robust_to_numeric(df.iloc[:, 24+1]) # 栏目24
        
        return df_final
    except Exception as e:
        print(f"    [错误] 处理文件 {file_path.name} 时发生未知错误。")
        print(traceback.format_exc())
        return None

# ==============================================================================
# 3. 主流程 (Transformation & Loading)
# ==============================================================================
def main():
    print("开始执行数据处理流程...")
    all_dataframes = []
    if not Config.INPUT_DIR.exists():
        print(f"[致命错误] 输入文件夹不存在: {Config.INPUT_DIR}")
        return
    for file_path in sorted(Config.INPUT_DIR.glob('*.xlsx')):
        if re.match(Config.FILENAME_PATTERN, file_path.name):
            df = process_single_file(file_path)
            if df is not None:
                all_dataframes.append(df)
    if not all_dataframes:
        print("\n处理结束，但未从任何文件中成功提取数据。请检查上述错误日志。")
        return
    print("\n所有文件数据读取完毕，正在聚合成总表...")
    long_df = pd.concat(all_dataframes, ignore_index=True)
    
    long_df.fillna(0, inplace=True)

    print("正在生成最终的多工作表Excel报表...")
    Config.OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)
    with pd.ExcelWriter(Config.OUTPUT_FILE, engine='openpyxl') as writer:
        sheets_to_create = Config.get_sheets_to_create()
        for sheet_name, value_col in sheets_to_create.items():
            print(f"  -> 正在生成工作表: {sheet_name}")
            try:
                pivot_df = long_df.pivot_table(
                    index=['序号', '行业'], columns='年月', values=value_col, aggfunc='sum')
                pivot_df.columns.name = None
                pivot_df = pivot_df.reset_index()
                pivot_df.rename(columns={'行业': '分类'}, inplace=True)
                pivot_df.to_excel(writer, sheet_name=sheet_name, index=False)
            except KeyError as e:
                print(f"    [警告] 在生成工作表 '{sheet_name}' 时找不到数据列 '{e}'，已跳过。")
                continue
    print(f"\n全部任务成功完成！结果已保存到: {Config.OUTPUT_FILE}")

if __name__ == '__main__':
    main()

开始执行数据处理流程...
  -> 正在处理文件: 201801.xlsx
  -> 正在处理文件: 201802.xlsx
  -> 正在处理文件: 201803.xlsx
  -> 正在处理文件: 201804.xlsx
  -> 正在处理文件: 201805.xlsx
  -> 正在处理文件: 201806.xlsx
  -> 正在处理文件: 201807.xlsx
  -> 正在处理文件: 201808.xlsx
  -> 正在处理文件: 201809.xlsx
  -> 正在处理文件: 201810.xlsx
  -> 正在处理文件: 201811.xlsx
  -> 正在处理文件: 201812.xlsx
  -> 正在处理文件: 201901.xlsx
  -> 正在处理文件: 201902.xlsx
  -> 正在处理文件: 201903.xlsx
  -> 正在处理文件: 201904.xlsx
  -> 正在处理文件: 201905.xlsx
  -> 正在处理文件: 201906.xlsx
  -> 正在处理文件: 201907.xlsx
  -> 正在处理文件: 201908.xlsx
  -> 正在处理文件: 201909.xlsx
  -> 正在处理文件: 201910.xlsx
  -> 正在处理文件: 201911.xlsx
  -> 正在处理文件: 201912.xlsx
  -> 正在处理文件: 202001.xlsx
  -> 正在处理文件: 202002.xlsx
  -> 正在处理文件: 202003.xlsx
  -> 正在处理文件: 202004.xlsx
  -> 正在处理文件: 202005.xlsx
  -> 正在处理文件: 202006.xlsx
  -> 正在处理文件: 202007.xlsx
  -> 正在处理文件: 202008.xlsx
  -> 正在处理文件: 202009.xlsx
  -> 正在处理文件: 202010.xlsx
  -> 正在处理文件: 202011.xlsx
  -> 正在处理文件: 202012.xlsx
  -> 正在处理文件: 202101.xlsx
  -> 正在处理文件: 202102.xlsx
  -> 正在处理文件: 202103.xlsx
  -> 正在处理文件

d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


  -> 正在处理文件: 202403.xlsx
  -> 正在处理文件: 202404.xlsx
  -> 正在处理文件: 202405.xlsx
  -> 正在处理文件: 202406.xlsx
  -> 正在处理文件: 202407.xlsx
  -> 正在处理文件: 202408.xlsx
  -> 正在处理文件: 202409.xlsx
  -> 正在处理文件: 202410.xlsx
  -> 正在处理文件: 202411.xlsx
  -> 正在处理文件: 202412.xlsx


d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


  -> 正在处理文件: 202501.xlsx
  -> 正在处理文件: 202502.xlsx
  -> 正在处理文件: 202503.xlsx
  -> 正在处理文件: 202504.xlsx
  -> 正在处理文件: 202505.xlsx


d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")



所有文件数据读取完毕，正在聚合成总表...
正在生成最终的多工作表Excel报表...
  -> 正在生成工作表: 完成新装增容_容量
  -> 正在生成工作表: 完成新装_容量
  -> 正在生成工作表: 完成增容_容量
  -> 正在生成工作表: 完成新装增容_户数
  -> 正在生成工作表: 完成新装_户数
  -> 正在生成工作表: 完成增容_户数

全部任务成功完成！结果已保存到: E:\A智网\业扩分析\数据准备\10kV以上业扩.xlsx
